# Agent Example

In [1]:
# %pip install wikipedia openai chromadb langchain

In [2]:
# import os
# os.environ['OPENAI_API_KEY'] = ''

In [3]:
from groq import Groq
import os

In [5]:
from langchain_groq.chat_models import ChatGroq
llm = ChatGroq(api_key="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")

In [6]:
from langchain.agents import initialize_agent, AgentType, load_tools
# from langchain.chat_models import ChatOpenAI
from langchain_groq.chat_models import ChatGroq

llm = ChatGroq(api_key="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")
tools = load_tools(['wikipedia', "llm-math"], llm=llm)

agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    return_intermediate_steps=True,
    verbose=True,
)

In [7]:
from langchain.callbacks import StdOutCallbackHandler

question = """
What is the current American president's age? 
Return the age multiplied by 82
"""

handler = StdOutCallbackHandler()
response = agent(
    {"input": question}, 
    callbacks=[handler]
)

response



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
What is the current American president's age? 
Return the age multiplied by 82

Tho

{'input': "\nWhat is the current American president's age? \nReturn the age multiplied by 82\n",
 'output': 'Joe Biden is currently 81 years old, so his age multiplied by 82 is 6642.',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='Current American President\n', log='I need to find out who the current American president is and his age, then multiply that age by 82.\nAction: wikipedia\nAction Input: Current American President\n'),
   'Page: President of the United States\nSummary: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.\nThe power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in Am

In [8]:
response['intermediate_steps'][0][0]

AgentAction(tool='wikipedia', tool_input='Current American President\n', log='I need to find out who the current American president is and his age, then multiply that age by 82.\nAction: wikipedia\nAction Input: Current American President\n')

In [9]:
import wikipedia

wikipedia.search('current American president')

['President of the United States',
 'List of presidents of the United States',
 'List of current presidents of legislatures',
 'List of current vice presidents and designated acting presidents',
 'Abraham Lincoln',
 'George Washington',
 'Andrew Johnson',
 'Joe Biden',
 'Vice President of the United States',
 'Thomas Jefferson']

In [10]:
wikipedia.summary('President of the United States')

'The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.\nThe power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world\'s most powerful political figures and the leader of the world\'s only remaining superpower. As the leader of the nation with the largest economy by nominal GDP, the president possesses significant domestic and international hard and soft power. For much of the

In [11]:
wikipedia.summary('List of presidents of the United States')

'The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of individuals who have served as president.\nThe presidency of William Henry Harrison, who died 31 days after taking office in 1841, was the shortest in American history. Franklin D. Roosevelt served the longest, over twelve years, before dying early in his fourth term in 1945. He is the only U.S. president to have se

In [12]:
from langchain import LLMMathChain

chain = LLMMathChain.from_llm(llm, verbose=True)

In [13]:
chain.run('45 * 89', callbacks=[handler])



> Entering new LLMMathChain chain...
45 * 89

> Entering new LLMChain chain...
Prompt after formatting:
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: 45 * 89


> Finished chain.
```text
45 * 89
```
...numexpr.evaluate("45 * 89")...

Answer: 4005
> Finished chain.


'Answer: 4005'

# Custom Tools

In [14]:
from langchain.document_loaders import PyPDFLoader

file_path = 'G:\youtube_transcript\element_of_statistical_learning.pdf'

loader = PyPDFLoader(file_path=file_path)
data = loader.load_and_split()

data[:10]

[Document(page_content='Springer Series in Statistics\nTrevor Hastie\nRobert TibshiraniJerome FriedmanSpringer Series in Statistics\nThe Elements of\nStatistical Learning\nData Mining, Inference, and Prediction\nThe Elements of Statistical LearningDuring the past decade there has been an explosion in computation and information tech-\nnology. With it have come vast amounts of data in a variety of fields such as medicine, biolo-gy, finance, and marketing. The challenge of understanding these data has led to the devel-opment of new tools in the field of statistics, and spawned new areas such as data mining,machine learning, and bioinformatics. Many of these tools have common underpinnings butare often expressed with different terminology. This book describes the important ideas inthese areas in a common conceptual framework. While the approach is statistical, theemphasis is on concepts rather than mathematics. Many examples are given, with a liberaluse of color graphics. It should be a v

In [15]:
# !pip install tiktoken

In [18]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

docsearch = Chroma.from_documents(
    data, 
    embeddings, 
    collection_name="statistical_learning"
)

In [19]:
from langchain.chains import RetrievalQA

llm = ChatGroq(api_key="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")

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever()
)

In [20]:
chain.run('What is gaussian mixture model?', callbacks=[handler])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
13.3k-Nearest-Neighbor Classiﬁers 463
that they are deﬁned by algorithms, rather than optimizatio n of some ﬁxed
criteria; this makes it diﬃcult to understand their propert ies.
13.2.3 Gaussian Mixtures
The Gaussian mixture model can also bethought of as aprototy pe method,
similar in spirit to K-means and LVQ. We discuss Gaussian mixtures in
some detail in Sections 6.8, 8.5 and 12.7. Each cluster is des cribed in terms
of a Gaussian density, which has a centroid (as in K-means), and a covari-
ance matrix. The comparison becomes crisper if we restrict t he component
Gaussians to have a scalar covariance matrix (Exercise 13.1 ). The two steps
of the alternating EM

'A Gaussian mixture model is a type of clustering method that describes each cluster in terms of a Gaussian density, which has a centroid and a covariance matrix. It is similar to K-means in that it assigns observations to clusters based on their likelihood of belonging to each cluster. However, Gaussian mixture models are often referred to as a soft clustering method because they allow observations to belong to multiple clusters with varying degrees of membership, while K-means is a hard clustering method because it assigns observations to a single cluster. Gaussian mixture models can be used to represent the feature density in each class, which produces smooth posterior probabilities for classifying a given data point. They can also be used for density estimation and can be viewed as a kind of kernel method. The parameters of a Gaussian mixture model are typically fit using maximum likelihood and the EM algorithm.'

In [21]:
from langchain.agents import Tool

description = """
useful for when you need to answer questions 
about Machine Learning and Statistical Learning. 
Input should be a fully formed question.
"""

retrieval_tool = Tool(
    name="Statistical Learning Book",
    func=chain.run,
    description=description,
)

In [22]:
tools = load_tools(['wikipedia', "llm-math"], llm=llm)

tools.append(retrieval_tool)

agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    return_intermediate_steps=True,
)

In [23]:
question = """
Can we use machine learning to predict the age of the president?
"""

response = agent(
    {"input": question}, 
    callbacks=[handler]
)

response



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.
Statistical Learning Book: 
useful for when you need to answer questions 
about Machine Learning and Statistical Learning. 
Input should be a fully formed question.


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia, Calculator, Statistical Learning Book]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: 

{'input': '\nCan we use machine learning to predict the age of the president?\n',
 'output': "Yes, machine learning can be used to predict the age of a president based on certain features. However, it's important to consider the ethical implications and data availability before proceeding with such a task. Relevant features could include years in office, political party, age at inauguration, and others. The accuracy of the prediction would depend on the quality and relevance of the features used, as well as the size and representativeness of the training data.",
 'intermediate_steps': [(AgentAction(tool='Statistical Learning Book', tool_input='Can machine learning be used to predict the age of a president based on certain features?"\n', log='This question is asking about the possibility of using machine learning to predict the age of a president, but it doesn\'s specify which president or what data is available. To answer this question, I first need to clarify if we have data about a s

In [24]:
response['output']

"Yes, machine learning can be used to predict the age of a president based on certain features. However, it's important to consider the ethical implications and data availability before proceeding with such a task. Relevant features could include years in office, political party, age at inauguration, and others. The accuracy of the prediction would depend on the quality and relevance of the features used, as well as the size and representativeness of the training data."

In [25]:
question = """
What is a support vector machine?
"""

response = agent(
    {"input": question}, 
    callbacks=[handler]
)

response



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.
Statistical Learning Book: 
useful for when you need to answer questions 
about Machine Learning and Statistical Learning. 
Input should be a fully formed question.


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia, Calculator, Statistical Learning Book]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: 

{'input': '\nWhat is a support vector machine?\n',
 'output': 'A support vector machine (SVM) is a type of supervised machine learning model used for classification and regression analysis. It is based on statistical learning theory and is designed to find the maximum-margin hyperplane that best separates data into classes. SVMs can also handle non-linearly separable data using the kernel trick. The popularity of SVMs is likely due to their amenability to theoretical analysis and their flexibility in being applied to a wide variety of tasks.',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='Support vector machine\n', log='I need to find a definition or explanation of what a support vector machine (SVM) is. I can find this information on Wikipedia.\n\nAction: wikipedia\nAction Input: Support vector machine\n'),
   "Page: Support vector machine\nSummary: In machine learning, support vector machines (SVMs, also support vector networks) are supervised max-margin models w

In [26]:
response['output']

'A support vector machine (SVM) is a type of supervised machine learning model used for classification and regression analysis. It is based on statistical learning theory and is designed to find the maximum-margin hyperplane that best separates data into classes. SVMs can also handle non-linearly separable data using the kernel trick. The popularity of SVMs is likely due to their amenability to theoretical analysis and their flexibility in being applied to a wide variety of tasks.'